In [53]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from IPython.display import display
import seaborn as sns
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Filter data

In [54]:
def filter_data(data_sent , max_flux= -12):
    data = data_sent.copy()
    max_flux = -12
    min_flux = 26
    data = data[data['flux_aper']<max_flux]

    data = data[data['significance']>2]
    data_class = data[['class']]

    data_sig = data['significance']
    data_id = data['src_id']
    data_name = data['src_n']
    obs_info_params = [ 'livetime','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]
    data_val = data.drop(columns=obs_info_params)
    return data_val

def norm_data(data_sent):
    data = data_sent.copy()
    #data.replace()
    for d in data:
        max_val = np.amax(data[d])
        min_val =  np.amin(data[d])
        data[d] = (data[d]-min_val)/(max_val-min_val)
    return data
def std_data(data_sent):
    data = data_sent.copy()
    for d in data:
        mean =  np.mean(data[d])
        std = np.sqrt(np.var(data[d]))
        data[d] = (data[d]-mean)/std 
    return data
def do_nothing(data_sent):
    return data_sent

In [55]:
def extract_data(data_sent ,reduce_fn = ' '):
    data = data_sent.copy()
    data = data.sample(frac=1)
    data = filter_data(data)
    #display(data)
    data_id = data[[ 'class' ,'src_n' , 'src_id' ,'significance' , ]]
    data_id = data_id
    data_val = data.drop([ 'class' ,'src_n' , 'src_id' ,'significance' ,] , axis=1)
    data_val = reduce_fn(data_val)
    return data_val , data_id


In [56]:
def details(data_sent):
    sp = sum(train['flux_aper_b'].isna()) / (sum(~train['flux_aper_b'].isna()) + sum(train['flux_aper_b'].isna()))
    print('________________________________________________')
    print('Sparsity in the data : {:.2f}'.format(sp))
    num_rows = data_sent.shape
    print('Data shape' , num_rows)
    #display(data_sent['class'].value_counts())
    print('Number of sources : ')
    cl = data_sent['class'].unique()
    for c in cl:
        num_src = len(data_sent[data_sent['class']==c]['src_id'].unique())
        num_obs = len(data_sent[data_sent['class']==c])
        print(c ,' \t ' , num_src , '\t' , num_obs)

### RF imputer Missingpy

In [57]:
import sklearn.neighbors._base
from os import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [58]:
from missingpy import MissForest 

In [59]:
def rf_impute(d, i ):
    data = pd.concat([i , d] , axis=1)
    data = data.drop(columns=['src_n' , 'src_id' , 'significance' ,])
    rf_imputer = MissForest()
    #new_data = d.drop(columns= ['class'])
    new_data = rf_imputer.fit_transform(d)
    return new_data , rf_imputer

# Not combining observations

## Load data

Here we will impute each class seperately ,
for testing we can pass given sample through each imputer and then combine the result

In [60]:

train = pd.read_csv('../processed_data/'+'BH'+'_.csv' , index_col='obs_id')
df_bh = train.sample(frac=1)

train = pd.read_csv('../processed_data/'+'NS'+'_.csv' , index_col='obs_id')
df_ns = train.sample(frac=1)

train = pd.read_csv('../processed_data/'+'CV'+'_.csv' , index_col='obs_id')
df_cv = train.sample(frac=1)

train = pd.read_csv('../processed_data/'+'PULSAR'+'_.csv' , index_col='obs_id')
df_plsr = train.sample(frac=1)

train = pd.read_csv('../processed_data/'+'TUC'+'_.csv' , index_col='obs_id')
# df_tuc = train.sample(frac=1)
# src = pd.read_csv('../source_list/TUC_data_clean_source_list.csv')
# display(src.head())
# tuc_class = ['CV           ','MSP          ', 'QLX          ']
# src_list = src[src['A_SOURCE_TYPE'].isin(tuc_class)]['SRC_ID']
# df_tuc = df_tuc[df_tuc['src_id'].isin(src_list)]

train = pd.concat([df_bh , df_ns , df_cv, df_plsr ,])
details(train)

________________________________________________
Sparsity in the data : 0.26
Data shape (1870, 108)
Number of sources : 
BH  	  29 	 199
NS  	  52 	 440
CV  	  70 	 786
PULSAR  	  126 	 445


# Choose Features

In [67]:
sp = [] 
for f in feat_used:
    #print(f)
    na = train[feat_used][f].isna().value_counts()
    try:
        sp.append(1-na[0]/(na[0]+na[1]))
    except:
        sp.append(0)
sp =  np.asarray(sp)
sparsity = pd.DataFrame({
    "feat" : feat_used , 
    "sp_val" : sp
}).sort_values(by='sp_val' , ascending=False)
sparsity

,feat,sp_val
24,flux_aper_lolim_u,0.724064
38,flux_aper_u,0.702674
5,photflux_aper_lolim_u,0.654545
13,photflux_aper_u,0.631551
34,flux_aper_lolim_s,0.408021
...,...,...
60,likelihood,0.000000
1,src_id,0.000000
3,class,0.000000
2,significance,0.000000


In [71]:
sp_feat = sparsity[sparsity['sp_val']>0.5]
sp_feat

,feat,sp_val
24,flux_aper_lolim_u,0.724064
38,flux_aper_u,0.702674
5,photflux_aper_lolim_u,0.654545
13,photflux_aper_u,0.631551


In [61]:
from features import features as feat 
all_feat = list(feat['info'])+list(feat['flux']['photon'])+list(feat['flux']['energy'])+list(feat['variability'])+list(feat['hardness'])+list(feat['model_fit']['powerlaw'])+list(feat['model_fit']['bb'])+list(feat['model_fit']['brems'])+list(feat['info_pre_filter'])

In [62]:
feat_to_drop = list(feat['model_fit']['powerlaw']) + list(feat['model_fit']['bb']) + list(feat['model_fit']['brems']) 
#feat_to_drop = []
feat_used = [item for item in all_feat if item not in feat_to_drop]

Remove features which are highl sparse , sparisty level > 80

In [736]:
train_set = train[feat_used]

In [737]:
data_val , data_id   = extract_data(train_set ,  impute_fn= rf_impute , reduce_fn= do_nothing , rf_impute=True )

In [738]:
len(np.unique(data_id['src_id']))

317

In [739]:
data_val

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_456,-6.105573,-6.015473,NaN,-6.218963,-6.425506,-6.665144,-6.759201,-5.901010,-6.556737,NaN,...,0.35775,0.1805,0.5778,0.8126,0.4316,0.5990,0.0256,0.3923,0.6384,0.3173
CV_OBS_707,-4.255394,-3.674279,NaN,-4.317223,-4.165389,-4.182567,-4.266081,-3.636952,-4.212823,NaN,...,0.95720,-0.2130,-0.0281,0.0269,-0.1955,0.1380,-0.0631,-0.1199,-0.0849,0.0362
NS_OBS_51,-4.959793,-4.571541,NaN,-5.016148,-5.086875,-5.272866,-5.205582,-4.532096,-5.142185,NaN,...,NaN,0.0481,0.2380,0.4741,0.2705,0.3504,0.1330,0.1449,0.3748,0.2411
NS_OBS_401,-5.560352,-5.558305,NaN,-5.631527,-6.262728,-6.842241,-6.576426,-5.483068,-6.391795,NaN,...,0.69025,0.6065,0.7989,0.9888,0.8788,0.9038,0.4166,0.7083,0.9394,0.6883
PULSAR_OBS_118,-6.152736,-5.188827,NaN,NaN,-5.768530,-5.106016,-6.633764,-5.028260,-6.013900,NaN,...,NaN,-1.0000,-0.1893,-0.7876,-1.0000,-0.5166,-0.8838,-0.9994,-0.9994,-0.7258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_OBS_393,-6.287434,-5.591590,-5.624702,-6.484921,-5.994819,-5.770062,-6.289713,-5.480303,-6.117589,-6.721704,...,0.99940,-0.6552,-0.1118,-0.3829,-0.8001,-0.0556,-0.4691,-0.3935,-0.6096,-0.2723
TUC_OBS_400,-6.263524,-5.842241,NaN,-6.463820,-6.302161,-5.954677,-6.787546,-5.689094,-6.474049,NaN,...,0.41525,-0.3029,0.3904,-0.1068,-0.6515,-0.1618,-0.6565,0.0537,-0.3985,-0.4310
PULSAR_OBS_218,-6.238222,-4.902743,-5.928486,-6.371815,-5.487983,-5.160773,-5.582694,-4.871278,-5.531357,-6.354578,...,NaN,-0.8251,-0.6627,-0.8301,-0.9151,-0.3092,-0.4266,-0.7458,-0.8745,-0.3685


In [740]:
data_id 

,class,src_n,src_id,significance
obs_id,,,,
TUC_OBS_456,TUC,CXOGLB J002405.3-720421,TC0045,10.20
CV_OBS_707,CV,XMMSL2 J004231.9+41162,CV0110,174.76
NS_OBS_51,NS,1A 1744-361,NS0004,11.50
NS_OBS_401,NS,SAXWFC J1748.1-2446.,NS0072,19.47
PULSAR_OBS_118,PULSAR,PSR J0729-1448,PL0022,3.24
...,...,...,...,...
CV_OBS_393,CV,CXOGLB J002402.5-72051,CV0070,22.52
TUC_OBS_400,TUC,CXOGLB J002403.6-720459,TC0040,12.38
PULSAR_OBS_218,PULSAR,PSR J1744-1134,PL0080,14.80


In [741]:
new_data , random_forest_imputer = rf_impute(data_val, data_id)
#train_data.index.name = 'index'
#display(train_data.describe())
#train_data.to_csv('../processed_data/train_norm_rf_impute')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [742]:
data_val

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_456,-6.105573,-6.015473,NaN,-6.218963,-6.425506,-6.665144,-6.759201,-5.901010,-6.556737,NaN,...,0.35775,0.1805,0.5778,0.8126,0.4316,0.5990,0.0256,0.3923,0.6384,0.3173
CV_OBS_707,-4.255394,-3.674279,NaN,-4.317223,-4.165389,-4.182567,-4.266081,-3.636952,-4.212823,NaN,...,0.95720,-0.2130,-0.0281,0.0269,-0.1955,0.1380,-0.0631,-0.1199,-0.0849,0.0362
NS_OBS_51,-4.959793,-4.571541,NaN,-5.016148,-5.086875,-5.272866,-5.205582,-4.532096,-5.142185,NaN,...,NaN,0.0481,0.2380,0.4741,0.2705,0.3504,0.1330,0.1449,0.3748,0.2411
NS_OBS_401,-5.560352,-5.558305,NaN,-5.631527,-6.262728,-6.842241,-6.576426,-5.483068,-6.391795,NaN,...,0.69025,0.6065,0.7989,0.9888,0.8788,0.9038,0.4166,0.7083,0.9394,0.6883
PULSAR_OBS_118,-6.152736,-5.188827,NaN,NaN,-5.768530,-5.106016,-6.633764,-5.028260,-6.013900,NaN,...,NaN,-1.0000,-0.1893,-0.7876,-1.0000,-0.5166,-0.8838,-0.9994,-0.9994,-0.7258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_OBS_393,-6.287434,-5.591590,-5.624702,-6.484921,-5.994819,-5.770062,-6.289713,-5.480303,-6.117589,-6.721704,...,0.99940,-0.6552,-0.1118,-0.3829,-0.8001,-0.0556,-0.4691,-0.3935,-0.6096,-0.2723
TUC_OBS_400,-6.263524,-5.842241,NaN,-6.463820,-6.302161,-5.954677,-6.787546,-5.689094,-6.474049,NaN,...,0.41525,-0.3029,0.3904,-0.1068,-0.6515,-0.1618,-0.6565,0.0537,-0.3985,-0.4310
PULSAR_OBS_218,-6.238222,-4.902743,-5.928486,-6.371815,-5.487983,-5.160773,-5.582694,-4.871278,-5.531357,-6.354578,...,NaN,-0.8251,-0.6627,-0.8301,-0.9151,-0.3092,-0.4266,-0.7458,-0.8745,-0.3685


In [743]:
imp_data =  pd.DataFrame(new_data , columns = data_val.columns.to_list()  , index=data_val.index.to_list())
imp_data.index.name = 'obs_id'
normalized_df=(imp_data-imp_data.mean())/imp_data.std()
display(imp_data)
normalized_df.describe()

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_456,-6.105573,-6.015473,-6.644854,-6.218963,-6.425506,-6.665144,-6.759201,-5.901010,-6.556737,-7.658549,...,0.357750,0.1805,0.5778,0.8126,0.4316,0.5990,0.0256,0.3923,0.6384,0.3173
CV_OBS_707,-4.255394,-3.674279,-4.228807,-4.317223,-4.165389,-4.182567,-4.266081,-3.636952,-4.212823,-4.557238,...,0.957200,-0.2130,-0.0281,0.0269,-0.1955,0.1380,-0.0631,-0.1199,-0.0849,0.0362
NS_OBS_51,-4.959793,-4.571541,-5.418719,-5.016148,-5.086875,-5.272866,-5.205582,-4.532096,-5.142185,-6.177446,...,0.768073,0.0481,0.2380,0.4741,0.2705,0.3504,0.1330,0.1449,0.3748,0.2411
NS_OBS_401,-5.560352,-5.558305,-6.150679,-5.631527,-6.262728,-6.842241,-6.576426,-5.483068,-6.391795,-6.625766,...,0.690250,0.6065,0.7989,0.9888,0.8788,0.9038,0.4166,0.7083,0.9394,0.6883
PULSAR_OBS_118,-6.152736,-5.188827,-5.695356,-6.403807,-5.768530,-5.106016,-6.633764,-5.028260,-6.013900,-6.185980,...,0.897447,-1.0000,-0.1893,-0.7876,-1.0000,-0.5166,-0.8838,-0.9994,-0.9994,-0.7258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_OBS_393,-6.287434,-5.591590,-5.624702,-6.484921,-5.994819,-5.770062,-6.289713,-5.480303,-6.117589,-6.721704,...,0.999400,-0.6552,-0.1118,-0.3829,-0.8001,-0.0556,-0.4691,-0.3935,-0.6096,-0.2723
TUC_OBS_400,-6.263524,-5.842241,-5.968306,-6.463820,-6.302161,-5.954677,-6.787546,-5.689094,-6.474049,-6.350269,...,0.415250,-0.3029,0.3904,-0.1068,-0.6515,-0.1618,-0.6565,0.0537,-0.3985,-0.4310
PULSAR_OBS_218,-6.238222,-4.902743,-5.928486,-6.371815,-5.487983,-5.160773,-5.582694,-4.871278,-5.531357,-6.354578,...,0.745780,-0.8251,-0.6627,-0.8301,-0.9151,-0.3092,-0.4266,-0.7458,-0.8745,-0.3685


,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
count,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,...,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03,1.817000e+03
mean,-1.651611e-14,-3.146019e-15,-1.131511e-14,1.033748e-14,5.691772e-15,9.352025e-15,-4.720006e-15,-1.806663e-15,-1.795323e-14,1.571054e-14,...,4.145159e-15,-2.913343e-16,-1.360863e-15,-3.910527e-16,-4.497106e-17,-1.212263e-16,1.745073e-15,-1.603316e-16,2.932895e-17,-1.538792e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.271210e+00,-2.729601e+00,-1.660576e+00,-1.955221e+00,-2.503148e+00,-2.450256e+00,-2.056559e+00,-2.595650e+00,-1.870617e+00,-2.157015e+00,...,-2.731093e+00,-1.709581e+00,-2.841861e+00,-1.990537e+00,-1.172985e+00,-2.198705e+00,-1.273944e+00,-1.960512e+00,-1.364656e+00,-1.406726e+00
25%,-6.840050e-01,-7.302926e-01,-7.461542e-01,-7.184004e-01,-7.231622e-01,-7.252288e-01,-7.819033e-01,-7.535267e-01,-6.912682e-01,-6.545270e-01,...,-6.193412e-01,-6.468881e-01,-6.329689e-01,-7.887632e-01,-8.840553e-01,-7.067174e-01,-8.049130e-01,-5.481603e-01,-8.229762e-01,-6.574810e-01
50%,-3.308781e-02,-5.341345e-02,-1.290179e-01,-5.466122e-02,-1.454944e-01,-1.615517e-01,-1.065050e-01,-1.437142e-01,-1.861366e-01,-1.668001e-01,...,2.795649e-01,-2.163276e-03,-8.350863e-02,-1.002358e-01,-1.516331e-01,-1.596551e-01,-1.155556e-01,-1.290072e-02,-4.371235e-02,-4.565482e-02
75%,6.358821e-01,6.629962e-01,8.285144e-01,6.275106e-01,5.919879e-01,5.605785e-01,6.956157e-01,6.036234e-01,6.162056e-01,6.943287e-01,...,8.720599e-01,5.695702e-01,9.380542e-01,9.420015e-01,5.685740e-01,5.810255e-01,5.907843e-01,8.266732e-01,7.522163e-01,4.177357e-01
max,2.398196e+00,2.228195e+00,3.014878e+00,2.401043e+00,2.508173e+00,2.681630e+00,2.268844e+00,2.484032e+00,2.505921e+00,3.176843e+00,...,1.033170e+00,2.127430e+00,1.330147e+00,1.446852e+00,2.080325e+00,1.956332e+00,2.929907e+00,1.372438e+00,1.583097e+00,2.215018e+00


In [744]:
data_id

,class,src_n,src_id,significance
obs_id,,,,
TUC_OBS_456,TUC,CXOGLB J002405.3-720421,TC0045,10.20
CV_OBS_707,CV,XMMSL2 J004231.9+41162,CV0110,174.76
NS_OBS_51,NS,1A 1744-361,NS0004,11.50
NS_OBS_401,NS,SAXWFC J1748.1-2446.,NS0072,19.47
PULSAR_OBS_118,PULSAR,PSR J0729-1448,PL0022,3.24
...,...,...,...,...
CV_OBS_393,CV,CXOGLB J002402.5-72051,CV0070,22.52
TUC_OBS_400,TUC,CXOGLB J002403.6-720459,TC0040,12.38
PULSAR_OBS_218,PULSAR,PSR J1744-1134,PL0080,14.80


In [745]:
processed_data_all = pd.concat([data_id , normalized_df] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+'all_once'+'_rf_impute.csv')

processed_data_all = pd.concat([data_id , imp_data] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+'all_once'+'_rf_impute_no_norm.csv')

In [694]:
processed_data_all = pd.concat([data_id , normalized_df] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+obj_class+'_rf_impute.csv')

In [695]:
processed_data_all = pd.concat([data_id , imp_data] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+obj_class+'_rf_impute_no_norm.csv')

In [696]:
processed_data_all

,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_203,PULSAR,PSR J1714-3810,PL0073,29.01,-3.734946,-3.520281,-4.532392,-3.751291,-4.395558,-5.438422,...,0.738005,0.627700,0.674000,0.980000,0.960000,0.907600,0.817600,0.650800,0.969400,0.864500
PULSAR_OBS_247,PULSAR,PSR J1745-2900,PL0081,107.91,-4.556111,-4.568315,-4.670717,-4.627456,-6.318487,-5.690796,...,0.750000,0.960000,1.000000,0.975000,0.832600,-0.421600,-1.000000,0.999400,0.911900,-0.999400
PULSAR_OBS_259,PULSAR,PSR J1745-2900,PL0081,107.91,-4.758454,-4.707301,-5.180333,-4.800519,-6.814175,-6.726999,...,0.750000,0.981300,1.000000,1.000000,0.977500,0.258000,-1.000000,0.999400,0.999400,-0.451000
PULSAR_OBS_224,PULSAR,PSR J1745-2900,PL0081,107.91,-4.892790,-4.913996,-5.137785,-4.968592,-6.675512,-6.168386,...,0.750000,0.962500,1.000000,0.985000,0.875100,-0.325400,-1.000000,0.999400,0.938200,-0.999400
PULSAR_OBS_393,PULSAR,PSR J1856+0113,PL0108,15.97,-5.317494,-4.959398,-6.475055,-5.371509,-5.430860,-5.769040,...,0.828200,0.043100,0.214200,0.592800,0.414100,0.499100,0.302900,0.129900,0.507200,0.402200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_183,PULSAR,PSR J1628-3205,PL0064,11.99,-5.060780,-4.629672,-5.407712,-5.116623,-5.272377,-5.019270,...,0.689148,0.154300,0.346700,0.051800,-0.143000,-0.200500,-0.385400,0.251100,-0.047500,-0.294800
PULSAR_OBS_242,PULSAR,PSR J1745-2900,PL0081,107.91,-4.658368,-4.695294,-4.684639,-4.738499,-6.320663,-5.707301,...,0.750000,0.950000,1.000000,0.968800,0.788900,-0.424100,-1.000000,0.999400,0.896900,-0.999400
PULSAR_OBS_20,PULSAR,PSR J0101-6422,PL0004,4.89,-5.781990,-4.979225,-5.071989,-5.987585,-5.437469,-4.934794,...,0.687307,-0.688900,-0.174300,-0.676500,-0.908800,-0.399100,-0.691400,-0.444700,-0.809500,-0.557200


In [587]:
import joblib 
joblib.dump(random_forest_imputer , 'forest/v3/'+obj_class+'_imp.joblib' , compress=0)

['forest/v3/BH_imp.joblib']

# 47 TUC application

In [588]:
trial = pd.read_csv('../processed_data/CV_.csv')
trial

,index,obs_id,class,src_n,src_id,livetime,significance,likelihood,pileup_flag,mstr_sat_src_flag,...,bb_ampl_lolim,bb_ampl_hilim,bb_stat,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
0,0,CV_OBS_0,CV,[DSH2013] 24,CV0001,9841.8,9.81,250.012,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,CV_OBS_1,CV,[DSH2013] 24,CV0001,66285.9,9.81,250.012,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,CV_OBS_2,CV,[DSH2013] 24,CV0001,52365.0,9.81,250.012,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,CV_OBS_3,CV,[HPH2013] 100,CV0002,4903.5,37.03,4392.422,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,CV_OBS_4,CV,[HPH2013] 100,CV0002,4108.6,37.03,4392.422,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,781,CV_OBS_781,CV,XMMSL2 J004231.9+41162,CV0110,3907.5,174.76,172643.300,False,False,...,0.000228,0.000440,2.256,4.334,6.475,3.119,5.096,13.680,NaN,0.465
782,782,CV_OBS_782,CV,XMMSL2 J004231.9+41162,CV0110,3907.5,174.76,172643.300,False,False,...,0.000159,0.000250,1.666,6.150,10.270,4.217,18.220,28.940,8.972,0.861
783,783,CV_OBS_783,CV,XMMSL2 J004231.9+41162,CV0110,3907.5,174.76,172643.300,False,False,...,0.000225,0.000392,2.444,6.490,12.520,4.064,5.549,14.910,NaN,0.942
784,784,CV_OBS_784,CV,XMMSL2 J004231.9+41162,CV0110,3974.2,174.76,172643.300,False,False,...,0.000348,0.000713,2.843,4.340,6.583,3.066,6.158,14.240,NaN,0.847


In [589]:
tuc = pd.read_csv('../processed_data/TUC_.csv' ,index_col='obs_id')
tuc = tuc[feat_used]

In [590]:
data_val , data_id   = extract_data(tuc,  impute_fn= rf_impute , reduce_fn= do_nothing , rf_impute=True )

In [591]:
data_id

,class,src_n,src_id,significance
obs_id,,,,
TUC_OBS_1209,TUC,CXOGLB J002415.9-720451,TC0122,7.89
TUC_OBS_1065,TUC,CXOGLB J002407.8-720441,TC0108,24.52
TUC_OBS_1828,TUC,CXOGLB J002404.4-720432,TC0199,4.23
TUC_OBS_1401,TUC,CXOGLB J002353.4-720511,TC0148,5.20
TUC_OBS_2352,TUC,CXOGLB J002434.1-720333,TC0270,4.83
...,...,...,...,...
TUC_OBS_1547,TUC,CXOGLB J002359.3-720448,TC0168,3.42
TUC_OBS_976,TUC,CXOGLB J002406.3-720452,TC0101,7.20
TUC_OBS_1061,TUC,CXOGLB J002407.8-720441,TC0108,24.52


In [592]:
data_val

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,-6.995249,-6.667360,NaN,NaN,-7.145998,-6.187822,NaN,-6.294906,NaN,NaN,...,0.441000,-0.2367,1.0000,-0.6202,-1.0000,-0.7127,-1.0000,0.9981,-0.9994,-0.9994
TUC_OBS_1065,-5.852633,-5.183626,-5.766750,-5.926648,-5.672028,-5.651306,-5.788346,-5.142969,-5.726304,-5.942714,...,0.684400,-0.3304,-0.1131,-0.1355,-0.3567,0.0681,-0.1193,-0.2224,-0.2473,-0.0262
TUC_OBS_1828,NaN,NaN,NaN,NaN,-7.483861,-6.995679,NaN,NaN,NaN,NaN,...,0.222000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TUC_OBS_1401,-5.405718,-6.922269,NaN,-5.695725,-5.992679,-5.759201,NaN,-5.579879,NaN,NaN,...,0.489400,0.3816,1.0000,1.0000,0.1480,0.1268,-1.0000,0.9994,0.9994,-0.9994
TUC_OBS_2352,-6.668978,-6.898253,NaN,-6.927750,-6.844360,-7.109970,-7.764977,-6.565910,-7.096042,NaN,...,0.555500,-0.1755,0.6777,0.9225,0.1818,0.7951,-0.0993,0.2948,0.6408,0.4147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,-5.852324,NaN,NaN,NaN,-6.229737,-6.053499,NaN,NaN,NaN,NaN,...,0.103250,0.0606,1.0000,1.0000,-0.1418,0.1718,-1.0000,0.9994,0.9994,-0.9994
TUC_OBS_976,-6.321937,-5.848630,-5.424350,-6.524329,-6.230918,-5.985900,-6.644357,-5.706859,-6.384787,-5.994391,...,0.583600,-0.4791,0.2030,-0.1330,-0.7039,-0.0269,-0.5528,-0.1387,-0.4397,-0.3086
TUC_OBS_1061,-5.945770,-5.403403,-5.691863,-6.078938,-5.903090,-5.717831,-6.153663,-5.318487,-6.007535,NaN,...,0.684400,-0.2929,0.1468,-0.0893,-0.4941,-0.0394,-0.4016,-0.0687,-0.2973,-0.2274


In [593]:
tuc_imp =  random_forest_imputer.transform(data_val)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


In [594]:
imp_data =  pd.DataFrame(tuc_imp , columns = data_val.columns.to_list()  , index=data_val.index.to_list())
imp_data.index.name = 'obs_id'
normalized_df=(imp_data-imp_data.mean())/imp_data.std()
display(imp_data)
normalized_df.describe()

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,-6.995249,-6.667360,-5.673662,-7.282439,-7.145998,-6.187822,-8.683497,-6.294906,-7.444246,-6.047718,...,0.441000,-0.236700,1.000000,-0.620200,-1.000000,-0.712700,-1.000000,0.998100,-0.999400,-0.99940
TUC_OBS_1065,-5.852633,-5.183626,-5.766750,-5.926648,-5.672028,-5.651306,-5.788346,-5.142969,-5.726304,-5.942714,...,0.684400,-0.330400,-0.113100,-0.135500,-0.356700,0.068100,-0.119300,-0.222400,-0.247300,-0.02620
TUC_OBS_1828,-7.222454,-8.289872,-6.767348,-7.258384,-7.483861,-6.995679,-8.583478,-7.180999,-7.500511,-7.961115,...,0.222000,-0.618144,0.497597,0.749703,-0.325799,0.414458,-0.677629,0.099233,0.229378,-0.14614
TUC_OBS_1401,-5.405718,-6.922269,-4.842619,-5.695725,-5.992679,-5.759201,-7.352701,-5.579879,-6.201074,-5.982962,...,0.489400,0.381600,1.000000,1.000000,0.148000,0.126800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_2352,-6.668978,-6.898253,-6.699890,-6.927750,-6.844360,-7.109970,-7.764977,-6.565910,-7.096042,-7.894349,...,0.555500,-0.175500,0.677700,0.922500,0.181800,0.795100,-0.099300,0.294800,0.640800,0.41470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,-5.852324,-6.898908,-5.115828,-6.112543,-6.229737,-6.053499,-7.505714,-5.985600,-6.409019,-6.063292,...,0.103250,0.060600,1.000000,1.000000,-0.141800,0.171800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_976,-6.321937,-5.848630,-5.424350,-6.524329,-6.230918,-5.985900,-6.644357,-5.706859,-6.384787,-5.994391,...,0.583600,-0.479100,0.203000,-0.133000,-0.703900,-0.026900,-0.552800,-0.138700,-0.439700,-0.30860
TUC_OBS_1061,-5.945770,-5.403403,-5.691863,-6.078938,-5.903090,-5.717831,-6.153663,-5.318487,-6.007535,-6.113837,...,0.684400,-0.292900,0.146800,-0.089300,-0.494100,-0.039400,-0.401600,-0.068700,-0.297300,-0.22740


,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
count,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,...,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03,1.572000e+03
mean,-2.469272e-14,4.565192e-16,7.512227e-15,5.460150e-15,-1.565047e-14,2.635155e-15,3.948213e-15,-6.917848e-15,1.339613e-14,-3.377564e-15,...,-2.237396e-15,4.022793e-16,4.519992e-17,-2.033996e-17,-3.299594e-16,1.129998e-17,-1.083668e-15,1.807997e-16,-5.875990e-16,7.299787e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.406206e+00,-2.503640e+00,-1.141165e+00,-1.016288e+00,-1.723163e+00,-2.206655e+00,-1.422608e+00,-1.858092e+00,-1.223375e+00,-1.284657e+00,...,-1.584285e+00,-1.327370e+00,-2.674993e+00,-1.785650e+00,-8.517296e-01,-1.902294e+00,-1.028907e+00,-1.483320e+00,-9.704444e-01,-1.130985e+00
25%,-8.172971e-01,-6.199410e-01,-1.009569e+00,-9.521547e-01,-7.475174e-01,-7.082876e-01,-8.366267e-01,-7.124169e-01,-8.439383e-01,-1.158092e+00,...,-8.080919e-01,-1.059949e+00,-7.481341e-01,-7.995909e-01,-8.517296e-01,-7.704298e-01,-1.028907e+00,-7.583028e-01,-9.704444e-01,-1.130985e+00
50%,-1.723959e-01,-3.517037e-02,-6.684748e-02,-2.263875e-01,-7.747876e-02,-1.087651e-01,-3.551376e-02,-9.797159e-02,-9.073497e-02,1.912642e-01,...,-1.481029e-01,1.385177e-01,-6.849133e-03,-1.864599e-01,-3.676439e-01,-1.610397e-01,-1.878280e-01,5.604722e-02,-1.866022e-01,-3.089515e-02
75%,6.449466e-01,5.407618e-01,6.064481e-01,6.444137e-01,6.530419e-01,5.990991e-01,6.332107e-01,5.451257e-01,6.078147e-01,7.152393e-01,...,6.957261e-01,7.120526e-01,1.230724e+00,6.789790e-01,6.468746e-01,5.791401e-01,5.710943e-01,8.897656e-01,6.248011e-01,5.008279e-01
max,3.574847e+00,3.357384e+00,4.076501e+00,3.833117e+00,3.726249e+00,4.129929e+00,3.135303e+00,3.829740e+00,3.777871e+00,3.391707e+00,...,2.013366e+00,2.721112e+00,1.263565e+00,1.848110e+00,3.440715e+00,2.229043e+00,4.055806e+00,1.322011e+00,2.137557e+00,2.806800e+00


In [595]:
display(normalized_df)

,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,photflux_aper_lolim_m,photflux_aper_b,photflux_aper_m,photflux_aper_lolim_u,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,-0.777697,-0.345741,0.630467,-0.976416,-0.968139,0.032185,-1.349116,-0.446694,-1.109773,0.779790,...,0.002279,0.282112,1.263565,-1.204263,-0.851730,-1.492485,-1.028907,1.320186,-0.970444,-1.130985
TUC_OBS_1065,0.961072,1.381398,0.486118,1.121150,1.323963,0.922682,1.637681,1.286317,1.491368,0.889878,...,0.877947,0.084538,-1.173485,-0.291113,0.597711,0.204116,1.604964,-0.392794,0.199021,0.786292
TUC_OBS_1828,-1.123444,-2.234426,-1.065474,-0.939199,-1.493535,-1.308674,-1.245930,-1.779760,-1.194965,-1.226250,...,-0.785607,-0.522195,0.163591,1.376564,0.667335,0.956718,-0.064806,0.058620,0.940224,0.550001
TUC_OBS_1401,1.641162,-0.642468,1.919134,1.478415,0.825334,0.743600,0.023806,0.629016,0.772517,0.847682,...,0.176405,1.585850,1.263565,1.848110,1.734868,0.331666,-1.028907,1.322011,2.137557,-1.130985
TUC_OBS_2352,-0.281196,-0.614512,-0.960869,-0.427671,-0.499077,-1.498373,-0.401520,-0.854401,-0.582557,-1.156250,...,0.414210,0.411158,0.557913,1.702104,1.811024,1.783815,1.664777,0.333100,1.579958,1.654897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,0.961543,-0.615275,1.495479,0.833550,0.456695,0.255132,-0.134051,0.018637,0.457668,0.763463,...,-1.212828,0.908994,1.263565,1.848110,1.081910,0.429446,-1.028907,1.322011,2.137557,-1.130985
TUC_OBS_976,0.246911,0.607301,1.017065,0.196469,0.454859,0.367331,0.754573,0.437984,0.494358,0.835700,...,0.515304,-0.229009,-0.481407,-0.286403,-0.184577,-0.002309,0.308515,-0.275321,-0.100148,0.229943
TUC_OBS_1061,0.819341,1.125567,0.602243,0.885540,0.964649,0.812264,1.260800,1.022262,1.065556,0.710470,...,0.877947,0.163610,-0.604453,-0.204075,0.288131,-0.029471,0.760702,-0.177075,0.121275,0.389913


In [596]:
obj_class

'BH'

In [597]:
processed_data_all = pd.concat([data_id , normalized_df] , axis=1)
processed_data_all.to_csv('../processed_data/v3/47_tuc_'+obj_class+'_rf_impute.csv')

In [598]:
processed_data_all = pd.concat([data_id , imp_data] , axis=1)
processed_data_all.to_csv('../processed_data/v3/47_tuc_'+obj_class+'_rf_impute_no_norm.csv')

In [599]:
processed_data_all

,class,src_n,src_id,significance,photflux_aper_hilim_h,photflux_aper_lolim_b,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_m,photflux_aper_hilim_s,...,var_inter_prob,hard_hm_lolim,hard_hm_hilim,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,hard_hm,hard_hs,hard_ms
obs_id,,,,,,,,,,,,,,,,,,,,,
TUC_OBS_1209,TUC,CXOGLB J002415.9-720451,TC0122,7.89,-6.995249,-6.667360,-5.673662,-7.282439,-7.145998,-6.187822,...,0.441000,-0.236700,1.000000,-0.620200,-1.000000,-0.712700,-1.000000,0.998100,-0.999400,-0.99940
TUC_OBS_1065,TUC,CXOGLB J002407.8-720441,TC0108,24.52,-5.852633,-5.183626,-5.766750,-5.926648,-5.672028,-5.651306,...,0.684400,-0.330400,-0.113100,-0.135500,-0.356700,0.068100,-0.119300,-0.222400,-0.247300,-0.02620
TUC_OBS_1828,TUC,CXOGLB J002404.4-720432,TC0199,4.23,-7.222454,-8.289872,-6.767348,-7.258384,-7.483861,-6.995679,...,0.222000,-0.618144,0.497597,0.749703,-0.325799,0.414458,-0.677629,0.099233,0.229378,-0.14614
TUC_OBS_1401,TUC,CXOGLB J002353.4-720511,TC0148,5.20,-5.405718,-6.922269,-4.842619,-5.695725,-5.992679,-5.759201,...,0.489400,0.381600,1.000000,1.000000,0.148000,0.126800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_2352,TUC,CXOGLB J002434.1-720333,TC0270,4.83,-6.668978,-6.898253,-6.699890,-6.927750,-6.844360,-7.109970,...,0.555500,-0.175500,0.677700,0.922500,0.181800,0.795100,-0.099300,0.294800,0.640800,0.41470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUC_OBS_1547,TUC,CXOGLB J002359.3-720448,TC0168,3.42,-5.852324,-6.898908,-5.115828,-6.112543,-6.229737,-6.053499,...,0.103250,0.060600,1.000000,1.000000,-0.141800,0.171800,-1.000000,0.999400,0.999400,-0.99940
TUC_OBS_976,TUC,CXOGLB J002406.3-720452,TC0101,7.20,-6.321937,-5.848630,-5.424350,-6.524329,-6.230918,-5.985900,...,0.583600,-0.479100,0.203000,-0.133000,-0.703900,-0.026900,-0.552800,-0.138700,-0.439700,-0.30860
TUC_OBS_1061,TUC,CXOGLB J002407.8-720441,TC0108,24.52,-5.945770,-5.403403,-5.691863,-6.078938,-5.903090,-5.717831,...,0.684400,-0.292900,0.146800,-0.089300,-0.494100,-0.039400,-0.401600,-0.068700,-0.297300,-0.22740
